In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import Bounds, linprog
np.set_printoptions(suppress=True)

### Задача 1

In [20]:
C = np.array([list(map(int, input(f'Введите {i+1} строку матрицы C:\n').split())) for i in range(3)])
print('\nМатрица C:')
print(C)

Введите 1 строку матрицы C:
12 5 11 6 0
Введите 2 строку матрицы C:
11 15 21 10 0
Введите 3 строку матрицы C:
13 15 15 11 0

Матрица C:
[[12  5 11  6  0]
 [11 15 21 10  0]
 [13 15 15 11  0]]


In [3]:
a1, a2, a3 = map(int, input('Введите через пробел числа a1, a2, a3:\n').split())

Введите через пробел числа a1, a2, a3:
100 110 40


In [4]:
A = np.array([a1, a2, a3])

In [21]:
b1, b2, b3, b4, b5 = map(int, input('Введите через пробел числа b1, b2, b3, b4, b5:\n').split())

Введите через пробел числа b1, b2, b3, b4, b5:
100 50 20 30 50


In [22]:
B = np.array([b1, b2, b3, b4, b5])

In [23]:
N = len(A)
M = len(B)

In [24]:
df = pd.DataFrame(C, columns=B, index=A)
df.columns.name = 'aᵢ \\ bⱼ'
df

aᵢ \ bⱼ,100,50,20,30,50
100,12,5,11,6,0
110,11,15,21,10,0
40,13,15,15,11,0


In [25]:
if sum(A) == sum(B):
    print('Задача закрытого типа')
else:
    print('Задача открытого типа')

Задача закрытого типа


In [26]:
P = np.zeros_like(C)
INF = 1000000
C_copy = C.copy()
A_copy = A.copy()
B_copy = B.copy()
i = 0
j = 0

In [27]:
while not np.all(C_copy == INF):
    i, j = np.unravel_index(C_copy.argmin(), C_copy.shape)
    P[i][j] = min(A_copy[i], B_copy[j])
    if A_copy[i] <= B_copy[j]:
        C_copy[i].fill(INF)
        B_copy[j] -= A_copy[i]
        A_copy[i] = 0
    else:
        C_copy[:, j].fill(INF)
        A_copy[i] -= B[j]
        B_copy[j] = 0  

In [28]:
if np.count_nonzero(P) == N + M - 1:
    print('Опорный план является невырожденным')
else:
    print('Опорный план является вырожденным')

Опорный план является вырожденным


In [29]:
F = 0
for i in range(N):
    for j in range(M):
        F += C[i][j] * P[i][j]
print('Полученный план перевозки груза:\n', P)
print('Значение целевой функции для этого опорного плана', F)

Полученный план перевозки груза:
 [[  0  50   0   0  50]
 [ 80   0   0  30   0]
 [ 20 -60   0   0   0]]
Значение целевой функции для этого опорного плана 790


In [30]:
A_u = np.array([[1 if i in range(j*M, M*(j+1)) else 0 for i in range(N*M)] for j in range(N)])
b_u = A

In [31]:
b_eq = B
A_eq = np.array([[1 if i % M == j else 0 for i in range(N*M)] for j in range(M)])

In [32]:
res = linprog(C.flatten(), A_u, b_u, A_eq, b_eq, options={'disp': True})

X = res.x.round(3).reshape(N, M)
F_min = res.fun.round(3)

res

Primal Feasibility  Dual Feasibility    Duality Gap         Step             Path Parameter      Objective          
1.0                 1.0                 1.0                 -                1.0                 145.0               
0.2280252163036     0.2280252163036     0.2280252163036     0.7849358282868  0.2280252163036     432.0744250764      
0.060738626895      0.06073862689497    0.06073862689497    0.7628818329975  0.06073862689524    963.2957724274      
0.008643753546843   0.008643753546842   0.008643753546842   0.8848564877981  0.008643753546762   1536.240395159      
0.0001187365411612  0.0001187365411612  0.0001187365411615  0.9864488229169  0.0001187365411587  1746.674462369      
6.040937716218e-09  6.040937782867e-09  6.040937686153e-09  0.999949132579   6.040941602264e-09  1749.999830375      
Optimization terminated successfully.
         Current function value: 1749.999830 
         Iterations: 5


     con: array([0.00001077, 0.00000522, 0.00000189, 0.000003  , 0.00000522])
     fun: 1749.9998303745895
 message: 'Optimization terminated successfully.'
     nit: 5
   slack: array([0.00001056, 0.00001165, 0.0000039 ])
  status: 0
 success: True
       x: array([ 0.00000047, 49.99999426, 19.99999752, 29.99999596,  0.00000123,
       99.99998865,  0.00000038,  0.00000022,  0.00000103,  9.99999808,
        0.00000011,  0.00000014,  0.00000037,  0.00000001, 39.99999547])

In [33]:
print('Оптимальный план перевозки груза X*:\n', X)
print('Оптимальная стоимость перевозки:\nF_min = F(X*) =', F_min)

Оптимальный план перевозки груза X*:
 [[  0.  50.  20.  30.   0.]
 [100.   0.   0.   0.  10.]
 [  0.   0.   0.   0.  40.]]
Оптимальная стоимость перевозки:
F_min = F(X*) = 1750.0


### Задача 2

In [38]:
matrix = pd.read_csv('data_ex.csv', delimiter=';', encoding='utf-8', header = None)
matrix

,0,1,2,3
0,12,10,9,11
1,14,6,5,8
2,12,8,8,4
3,10,5,9,7
4,7,10,9,11


In [40]:
np_matrix = matrix.values
np_matrix

array([[12, 10,  9, 11],
       [14,  6,  5,  8],
       [12,  8,  8,  4],
       [10,  5,  9,  7],
       [ 7, 10,  9, 11]], dtype=int64)

In [57]:
risk = np.zeros_like(matrix)
max_st = np.amax(np_matrix, axis=0)
max_st

np_array_risk1 = np.array([max_st[0]-np_matrix[0][0], 
                          max_st[0]-np_matrix[1][0], 
                          max_st[0]-np_matrix[2][0], 
                          max_st[0]-np_matrix[3][0],
                          max_st[0]-np_matrix[4][0]])
np_array_risk2 = np.array([max_st[1]-np_matrix[0][1], 
                          max_st[1]-np_matrix[1][1], 
                          max_st[1]-np_matrix[2][1], 
                          max_st[1]-np_matrix[3][1],
                          max_st[1]-np_matrix[4][1]])
np_array_risk3 = np.array([max_st[2]-np_matrix[0][2], 
                          max_st[2]-np_matrix[1][2], 
                          max_st[2]-np_matrix[2][2], 
                          max_st[2]-np_matrix[3][2],
                          max_st[2]-np_matrix[4][2]])
np_array_risk4 = np.array([max_st[3]-np_matrix[0][3], 
                          max_st[3]-np_matrix[1][3], 
                          max_st[3]-np_matrix[2][3], 
                          max_st[3]-np_matrix[3][3],
                          max_st[3]-np_matrix[4][3]])
  
risk[:, 0] = np_array_risk1
risk[:, 1] = np_array_risk2
risk[:, 2] = np_array_risk3
risk[:, 3] = np_array_risk4
risk

array([[2, 0, 0, 0],
       [0, 4, 4, 3],
       [2, 2, 1, 7],
       [4, 5, 0, 4],
       [7, 0, 0, 0]], dtype=int64)

In [68]:
max_a = np.amax(risk, axis=1)
max_a

array([2, 4, 7, 5, 7], dtype=int64)

In [86]:
strat = np.argmin(max_a)
if strat == 0:
        print("Стратегия 1 наиболее выгодная", np.amin(max_a), " - наименьший элемент")
if strat == 1:
        print("Стратегия 2 наиболее выгодная", np.amin(max_a), " - наименьший элемент")
if strat == 2:
        print("Стратегия 3 наиболее выгодная", np.amin(max_a), " - наименьший элемент")
if strat == 3:
        print("Стратегия 4 наиболее выгодная", np.amin(max_a), " - наименьший элемент")  
if strat == 4:
        print("Стратегия 5 наиболее выгодная", np.amin(max_a), " - наименьший элемент")    

Стратегия 1 наиболее выгодная 2  - наименьший элемент


In [81]:
modified_matrix0 = np_matrix*(-1)
modified_matrix = modified_matrix0 + np.amax(np_matrix)
modified_matrix

array([[ 2,  4,  5,  3],
       [ 0,  8,  9,  6],
       [ 2,  6,  6, 10],
       [ 4,  9,  5,  7],
       [ 7,  4,  5,  3]], dtype=int64)

In [82]:
risk2 = np.zeros_like(modified_matrix)
max_st = np.amax(modified_matrix, axis=0)
max_st

np_array_risk21 = np.array([max_st[0]-modified_matrix[0][0], 
                          max_st[0]-modified_matrix[1][0], 
                          max_st[0]-modified_matrix[2][0], 
                          max_st[0]-modified_matrix[3][0],
                          max_st[0]-modified_matrix[4][0]])
np_array_risk22 = np.array([max_st[1]-modified_matrix[0][1], 
                          max_st[1]-modified_matrix[1][1], 
                          max_st[1]-modified_matrix[2][1], 
                          max_st[1]-modified_matrix[3][1],
                          max_st[1]-modified_matrix[4][1]])
np_array_risk23 = np.array([max_st[2]-modified_matrix[0][2], 
                          max_st[2]-modified_matrix[1][2], 
                          max_st[2]-modified_matrix[2][2], 
                          max_st[2]-modified_matrix[3][2],
                          max_st[2]-modified_matrix[4][2]])
np_array_risk24 = np.array([max_st[3]-modified_matrix[0][3], 
                          max_st[3]-modified_matrix[1][3], 
                          max_st[3]-modified_matrix[2][3], 
                          max_st[3]-modified_matrix[3][3],
                          max_st[3]-modified_matrix[4][3]])
  
risk2[:, 0] = np_array_risk21
risk2[:, 1] = np_array_risk22
risk2[:, 2] = np_array_risk23
risk2[:, 3] = np_array_risk24
risk2

array([[5, 5, 4, 7],
       [7, 1, 0, 4],
       [5, 3, 3, 0],
       [3, 0, 4, 3],
       [0, 5, 4, 7]], dtype=int64)

In [85]:
max_a2 = np.amax(risk2, axis=1)
max_a2
strat = np.argmin(max_a2)
if strat == 0:
        print("Стратегия 1 наименее рискованная. ", np.amin(max_a2), " - наименьший элемент")
if strat == 1:
        print("Стратегия 2 наименее рискованная", np.amin(max_a2), "- наименьший элемент")
if strat == 2:
        print("Стратегия 3 наименее рискованная", np.amin(max_a2), "- наименьший элемент")
if strat == 3:
        print("Стратегия 4 наименее рискованная", np.amin(max_a2), "- наименьший элемент")  
if strat == 4:
        print("Стратегия 5 наименее рискованная", np.amin(max_a2), "- наименьший элемент")    

Стратегия 4 наименее рискованная 4 - наименьший элемент
